<a href="https://colab.research.google.com/github/khaoulamerah/Admin-univ-lib/blob/main/Final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
import pathlib

data_dir = pathlib.Path("/content/drive/MyDrive/Faces/train/")



In [ ]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

In [ ]:
happy = list(data_dir.glob('happy/*'))
PIL.Image.open(str(happy[0]))

In [ ]:
PIL.Image.open(str(happy[1]))

In [ ]:
sad = list(data_dir.glob('sad/*'))
PIL.Image.open(str(sad[0]))

In [ ]:
PIL.Image.open(str(sad[1]))

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
print(class_names)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(4):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
normalization_layer = layers.Rescaling(1./255)

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
# Save the trained model
model.save('/content/drive/MyDrive/emotion_recognition_model.h5')
print("Model saved successfully as '/content/drive/MyDrive/emotion_recognition_model.h5'")

In [ ]:

# Cell 2: Import required libraries for classification
# Add this cell for the classification functionality

from google.colab import output
from base64 import b64decode
from PIL import Image
import cv2
import numpy as np

# Define the emotion classes (matching your project structure)
EMOTION_CLASSES = ['angry', 'fear', 'happy', 'sad']

# Cell 3: Function to take photo and classify emotion
# Add this cell for the webcam classification function

def take_photo_and_classify_emotion(model, filename='photo.jpg', quality=0.8):
    """
    Take a photo using webcam and classify the emotion

    Args:
        model: Trained emotion recognition model
        filename: Name to save the captured photo
        quality: JPEG quality (0-1)
    """
    # Define JavaScript to open camera and take photo
    js = f'''
        async function takePhoto(quality) {{
          const div = document.createElement('div');
          const capture = document.createElement('button');
          capture.textContent = 'Take Photo for Emotion Recognition';
          capture.style.padding = '10px 20px';
          capture.style.fontSize = '16px';
          capture.style.backgroundColor = '#4CAF50';
          capture.style.color = 'white';
          capture.style.border = 'none';
          capture.style.borderRadius = '5px';
          capture.style.cursor = 'pointer';
          div.appendChild(capture);

          const video = document.createElement('video');
          video.style.display = 'block';
          video.style.width = '400px';
          video.style.height = '300px';
          video.style.border = '2px solid #333';
          video.style.borderRadius = '10px';

          const stream = await navigator.mediaDevices.getUserMedia({{video: true}});

          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          // Resize the output to fit the video element.
          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          // Wait for Capture to be clicked.
          await new Promise((resolve) => capture.onclick = resolve);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getVideoTracks()[0].stop();
          div.remove();
          return canvas.toDataURL('image/jpeg', quality);
        }}
        '''

    # Display JavaScript to take photo and capture it
    output.eval_js(js)

    # Retrieve and decode the photo
    data = output.eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)

    # Load the image and preprocess it for classification
    image = Image.open(filename)
    image = image.resize((180, 180))  # Match your model's input size
    image = np.array(image)
    image = image.astype('float32') / 255.0  # Normalize to [0,1]
    image = np.expand_dims(image, axis=0)

    # Use the model to classify the image
    predictions = model.predict(image)[0]

    # Get the predicted class
    predicted_class_index = np.argmax(predictions)
    predicted_emotion = EMOTION_CLASSES[predicted_class_index]
    confidence = predictions[predicted_class_index]

    # Display results
    print("\n" + "="*50)
    print("EMOTION RECOGNITION RESULTS")
    print("="*50)
    print(f"Predicted Emotion: {predicted_emotion.upper()}")
    print(f"Confidence: {confidence:.2%}")
    print("\nAll Emotion Probabilities:")

    # Sort emotions by probability (highest first)
    emotion_probs = list(zip(EMOTION_CLASSES, predictions))
    emotion_probs.sort(key=lambda x: x[1], reverse=True)

    for emotion, prob in emotion_probs:
        print(f"  {emotion.capitalize()}: {prob:.2%}")

    print("="*50)

    return predicted_emotion, confidence, predictions

# Cell 4: Function to classify existing image files
# Add this cell for classifying existing images

def classify_image_file(model, image_path, target_size=(180, 180)):
    """
    Classify emotion from an existing image file

    Args:
        model: Trained emotion recognition model
        image_path: Path to the image file
        target_size: Size to resize the image (width, height)
    """
    try:
        # Load and preprocess the image
        image = Image.open(image_path)
        image = image.resize(target_size)
        image = np.array(image)
        image = image.astype('float32') / 255.0
        image = np.expand_dims(image, axis=0)

        # Make prediction
        predictions = model.predict(image)[0]
        predicted_class_index = np.argmax(predictions)
        predicted_emotion = EMOTION_CLASSES[predicted_class_index]
        confidence = predictions[predicted_class_index]

        # Display results
        print(f"\nImage: {image_path}")
        print(f"Predicted Emotion: {predicted_emotion.upper()}")
        print(f"Confidence: {confidence:.2%}")

        return predicted_emotion, confidence, predictions

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None, None, None

# Cell 5: Test the classification with webcam
# Add this cell to test the webcam functionality

# Take a photo and classify emotion
print("Taking photo for emotion recognition...")
emotion, confidence, all_predictions = take_photo_and_classify_emotion(model)

# Cell 6: Test with existing images from your dataset
# Add this cell to test with your training images

# Example: Test with a happy image from your dataset
happy_images = list(data_dir.glob('happy/*.jpg'))
if happy_images:
    test_image_path = str(happy_images[0])
    print(f"Testing with image: {test_image_path}")
    classify_image_file(model, test_image_path)

# Cell 7: Load saved model (for future use)
# Add this cell if you want to load the saved model later

def load_saved_model(model_path='/content/drive/MyDrive/emotion_recognition_model.h5'):
    """Load the saved model"""
    try:
        loaded_model = tf.keras.models.load_model(model_path)
        print(f"Model loaded successfully from {model_path}")
        return loaded_model
    except Exception as e:
        print(f"Error loading model: {e}")
        return None

